\title{Testing of Sorensen XG LXI Programing without VISA with Python}
\author{Steven K Armour}
\maketitle

# Libraries used

In [1]:
#Libraries that are needed
import socket #library that will do the connection between computer and LXI inst
import time #used to pause program to handle non asynchronous actions
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pickle

# Base SCPI lxi program

In [2]:
class SupplySocket():
    """
    Prototype class for establishing computer to power supply connection via socket 
    (IP4) programming via the SCPI LXI protocol
    
    """
    def __init__(self, SupplyIP, SupplyPort):
        """
        initlizes the class instincince
        
        Args:
            SupplyIP (str): the IP address of the target supply
            SupplyPort (str): the socket port on the supply 
            ex Rigol DS scopes use 5555
        
        """
        #bind the IP and Port to the object
        self.SupplyIP=SupplyIP; self.SupplyPort=SupplyPort
    
    def SocketInit(self, TimeOutSec=3.0):
        """
        Method to startup the socket interconnect
        
        Args:
            None
            
        Note:
            good intro to sockets: 
            http://www.pythonforbeginners.com/code-snippets-source-code/port-scanner-in-python
        """
        # create socket instance in the class that is a streaming IP4
        self.s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # set socket options
        self.s.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 0)
        
        self.s.settimeout(TimeOutSec)

        
        #make the connection to the scope
        self.s.connect((self.SupplyIP, self.SupplyPort))
    
    def SocketClose(self):
        """
        Method for closeing the socket (IP4) connection
        Args:
            None
        """
        self.s.close()
        print('Scope Socket Closed')
    
    def Command(self, cmd):
        """
        Convince  Method to pass SCPI commands over the socket such that the 
        command is auto ended with a '\n' and is encoded to 
        Ascii bit and send on the socket connection. Most useful is sending
        commands and not quarries 
        
        Args:
            cmd (str): SCPI command to be sent needs to be in all caps
            example '*RST' will reset the supply; see the manual
        """
        cmd=f'{cmd}\n'
        self.s.sendall(cmd.encode())
    
    def Responce(self, resBitLen=12):
        """
        Convince Method for reciving the response of a quarry from the supply.
        This methods returns the raw retrived bit 
        
        Args:
            resBitLen (int): the N in 2^N to specify the returned informtion 
            package
        
        Returns:
            returns the response information from the supply as is
        """
        return self.s.recv(2**resBitLen)
    
    def Quarry(self, cmd, resBitLen=16, Ascii=True):
        """
        Convince Method for sending qurry (ask for -> receive) commands that
        will send the SCPI commands and automatically get the responce from
        the suply and will return the result 
        
        Args:
            cmd (str): SCPI command to be sent needs to be in all caps
            example '*IDN?' will ask the supply for its IDN number; 
            see the programing guide
            
            resBitLen (int): the N in 2^N to specify the returned informtion 
            package
            
            Ascii (bool:Default True): if True will try to return the response
            from the Quarry as a string assuming "utf-8" ascii decoding
        
        Returns:
            str from decoded response if Arg Ascii is True 
            
            raw response of Arg Ascii is False
        
        """
        #prep and send command
        cmd=f'{cmd}\n'
        self.s.sendall(cmd.encode())
        
        # get the response
        try:
            res=self.s.recv(2**resBitLen)
        except timeout:
            print('Responce Timed out')
    
        
        if Ascii:
            # attempt at byte to 'utf-8' decoding
            try:
                if res.decode('utf-8')=='command error':
                    print(f'Command: "{cmd}" is invalaid')
                else:
                    return res
            except:
                pass
        else:
            return res
            

    

## Modification for Sorensen issues 

In [3]:
class SorensenXGSocket(SupplySocket):
    pass

# Testing
The test was done with a Sorenson XG 600V 1.4A half rack DC power supply connected to a 10$\Omega$ load monitored with a voltmeter connected in parallel to the load. 

SCPI Command Summary  A-14 pg 306 Rev F of Manual 

## The Power Supply IP and Port

In [4]:
SupplyIP="192.168.2.5"
#Supnet:255, 255 0
SupplyPort = 5025

## Init the connection 

In [5]:
PS=SorensenXGSocket(SupplyIP, SupplyPort)
PS.SocketInit()

## Confirm Connection with IDN

In [6]:
PS.Quarry('*IDN?', 32)

b'AMETEK,XG 600-1.4, SN# 1729A'

IDN was returned incomplete, rerunning

In [7]:
PS.Quarry('*IDN?', 32)

b'00455, 3.01,11/06/14; ENET 06.50\r'

 again bad response 

In [13]:
PS.Quarry('*IDN?', 32)

b'AMETEK,XG 600-1.4, SN# 1729A00455, 3.01,11/06/14; ENET 06.50\r'

there appears to be an issue with buffer clearing. Need to find a way to clear the buffer before and after queries

## Trying to see if calling statues byte will solve above issue
See page 164 in Rev F manual

In [14]:
PS.Quarry('*STB?')

b'AMETEK,XG 600-1.4, SN# 1729A00455, 3.01,11/06/14; ENET 06.50\r'

Nope

In [15]:
PS.Quarry('*STB?')

b'4\r'

The second call yielded good response see table 5-10
will have to look into as means of dealing with the redundant/past response to new queries 

## Trying Setting output voltage and current 
for understanding of how setting the voltage and current effect the output of the supply see Automatic Mode Crossover pg 59 and Fig 3-3 on pg 59

In [22]:
PS.Command(':VOLT 10.0')

In [17]:
PS.Command(':CURR 1.4')

that worked

## Trying turning on and off the power to the load

In [18]:
PS.Command(':OUTP ON')

In [19]:
PS.Command(':OUTP OFF')

In [20]:
PS.Command(':OUTP ON')

that worked

## Get DC Supplies internal voltage and current readings

In [23]:
PS.Quarry('MEAS:VOLT?')

b'4\r'

past result issue

In [24]:
PS.Quarry('MEAS:VOLT?')

b'10.00903\r'

now gives correct output voltage confirmed by the parallel multimeter

In [25]:
PS.Quarry(':MEAS:CURR?')

b'10.00903\r'

past result issue

In [27]:
PS.Quarry(':MEAS:CURR?')

b'0.99824\r'

now gives correct current

# Results

These test indicated that the Sorensen XG DC power supply is easy to control with some issues with quarries. The Quarry issue will have to we resolved either through a specialty quarry check sequence; some unknown command to the socket to clear that socket connection register or some command to the power supply that clears its transmission buffer. Besides that issue using Python and Socket program proved rather easy and viable to control and shows that the need for LapView or a VISA program is overrated. 